In [ ]:
import panel as pn
import holoviews as hv
import numpy as np
import random
import pandas as pd

pn.extension("tabulator", "plotly", sizing_mode="stretch_width")
hv.extension("bokeh")

ACCENT_COLOR = "#0072B5"
THEME = pn.state.session_args.get('theme', [b'default'])[0].decode()

The `BeforeAfterSlider` 

The BeforeAfterSlider layout enables you to quickly compare two panels layed out on top of each other. This notebook will show how it works for different examples.

## Image

Source: http://www.nathanshipley.com/gan/#gan-015-toonify-layer-blending

In [ ]:
before_image = pn.pane.JPG("https://images.squarespace-cdn.com/content/v1/54f4d512e4b0f54e36c4976f/1600724128265-GKIIG185IDPOMXMHY17S/Jameela-Jamil-GQ-tout-2x3_01.jpg", embed=False, height=400, sizing_mode="scale_height")
after_image = pn.pane.JPG("https://images.squarespace-cdn.com/content/v1/54f4d512e4b0f54e36c4976f/1600724098968-JPWFSO2AN4QR6JI3A7YP/Cartoon_Jameela_Jamil.jpg", embed=False, height=400, sizing_mode="scale_height")

In [ ]:
image = pn.layout.BeforeAfterSlider(
        value=20, before=before_image, after=after_image, name="Image", height=410, width=400, sizing_mode="fixed"
)

In [ ]:
image_component = pn.Column(
    "# Image - nathanshipley.com ",
    pn.Row(pn.Row(pn.pane.Markdown("## After: Toonified", sizing_mode="fixed", width=400), pn.Spacer(sizing_mode="stretch_width"), pn.pane.Markdown("## Before: Original", sizing_mode="fixed", width=400))),
    image, width=500, sizing_mode="fixed"
)
image_component

## HvPlot

In [ ]:
import hvplot.pandas
from bokeh.sampledata import sea_surface_temperature as sst

df = sst.sea_surface_temperature
df2 = df.copy()
for _ in range(0,10):
    row=random.randint(0, len(df2)-1)
    df2.iloc[row,0]+=100

In [ ]:
before_hvplot=df.hvplot.heatmap(x='time.month', y='time.day', C='temperature', 
                  height=300, responsive=True, colorbar=False).aggregate(function=np.mean)
after_hvplot=df2.hvplot.heatmap(x='time.month', y='time.day', C='temperature', 
                  height=300, responsive=True, colorbar=False).aggregate(function=np.mean)

In [ ]:
hvplot_plot = pn.layout.BeforeAfterSlider(
        value=20, before=before_hvplot, after=after_hvplot, height=310, name="HvPlot"
)
hvplot_component = pn.Column(
    "# hvplot - Sea Surface Temperature",
    pn.Row(pn.Row(pn.pane.Markdown("## After: Randomized", sizing_mode="fixed", width=400), pn.Spacer(sizing_mode="stretch_width"), pn.pane.Markdown("## Before: Original", sizing_mode="fixed", width=400))),
    hvplot_plot,
)
hvplot_component

## Plotly

In [ ]:
import plotly.express as px

In [ ]:
df = px.data.election()
geojson = px.data.election_geojson()

In [ ]:
if THEME == "dark":
    plotly_template = "plotly_dark"
else:
    plotly_template = "plotly"

In [ ]:
def display_choropleth(candidate):
    fig = px.choropleth(
        df, geojson=geojson, color=candidate, title=candidate,
        locations="district", featureidkey="properties.district",
        projection="mercator", range_color=[0, 6500], template=plotly_template)
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(autosize = True)

    return fig

In [ ]:
plotly_before = pn.pane.Plotly(display_choropleth("Joly"), sizing_mode="stretch_both")
plotly_after = pn.pane.Plotly(display_choropleth("Coderre"), sizing_mode="stretch_both")

In [ ]:
plotly_plot = pn.layout.BeforeAfterSlider(
        value=20, before=plotly_before, after=plotly_after, height=310, name="Plotly"
)
plotly_component = pn.Column(
    "# Plotly - Election Data",
    pn.Row(pn.Row(pn.pane.Markdown("## After: Coderre", sizing_mode="fixed", width=200), pn.Spacer(sizing_mode="stretch_width"), pn.pane.Markdown("## Before: Joly", sizing_mode="fixed", width=200))),
    plotly_plot,
)
plotly_component

## Tabulator

In [ ]:
before_df = pd.DataFrame(np.random.randn(10, 5), columns=list('ABCDE'))
after_df = before_df.copy()

for _ in range(0,30):
    row=random.randint(0, len(after_df)-1)
    column=random.randint(0, len(after_df.columns)-1)
    after_df.iloc[row,column]+=1

In [ ]:
tabulator_before = pn.widgets.Tabulator(before_df)
tabulator_after = pn.widgets.Tabulator(after_df)

In [ ]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

tabulator_before.style.applymap(color_negative_red)
tabulator_after.style.applymap(color_negative_red);

In [ ]:
tabulator_table = pn.layout.BeforeAfterSlider(
        value=20, before=tabulator_before, after=tabulator_after, height=310, name="Tabulator"
)
tabulator_component = pn.Column(
    "# Tabulator - Random Data",
    pn.Row(pn.Row(pn.pane.Markdown("## After", sizing_mode="fixed", width=200), pn.Spacer(sizing_mode="stretch_width"), pn.pane.Markdown("## Before", sizing_mode="fixed", width=200))),
    tabulator_table,
    "* Negative Colors are red",
)
tabulator_component

## App

Lets wrap it up in a nice template

In [ ]:
pn.template.FastListTemplate(
    site="Panel", title="BeforeAfterSlider Examples",
    main=[
        image_component,
        hvplot_component,
        plotly_component,
        tabulator_component,
    ],
    accent_base_color=ACCENT_COLOR, header_background=ACCENT_COLOR
).servable();